In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
predcon = pd.read_csv('roadconditions.csv')
conds=pd.read_csv("roadconditions.csv",usecols=["label"])
conds=conds.values.tolist()
print(conds[312173])
init=[0,0,0,0,0]
out=[init[:val[0]-1]+val+init[val[0]:]for val in conds]
print(out[2])
predcon.head()


[4]
[1, 0, 0, 0, 0]


,Longitude,Latitude,Speed,Time,Acc X,Acc Y,Acc Z,gyro_x,gyro_y,gyro_z,label
0,72.023881,34.560316,22.0,55:18.5,-0.668003,0.483644,11.408719,-0.023986,-0.007432,-0.001248,1
1,72.023881,34.560316,22.0,55:18.5,-0.167599,-0.991230,11.504491,-0.055751,0.004174,-0.001248,1
2,72.023881,34.560316,22.0,55:18.5,-0.859545,-2.195550,9.182044,-0.083240,-0.009265,0.009748,1
3,72.023881,34.560316,22.0,55:18.5,-1.326428,0.907430,7.166065,-0.135163,-0.008043,0.001195,1
4,72.023881,34.560316,22.0,55:18.5,0.878699,1.194743,8.767835,-0.032538,-0.067297,0.000585,1


In [3]:
predcon.set_index('Time', inplace=True)

In [4]:
s1 = '55:18.5'
s2 = '55:18.7' # for example
FMT = '%M:%H.%S'
tdelta = datetime.strptime(s2, FMT) - datetime.strptime(s1, FMT)
print(tdelta)

0:00:02


In [5]:
df_final = predcon.drop(['Longitude', 'Latitude'], axis=1)

In [6]:
timestamps = df_final.index.unique()

In [7]:
X = np.array(df_final.drop(['label'], axis=1))

print(X[0])
y = np.array(out)

print(y[0])

[ 2.2000000e+01 -6.6800267e-01  4.8364350e-01  1.1408719e+01
 -2.3985624e-02 -7.4323980e-03 -1.2479980e-03]
[1 0 0 0 0]


In [8]:
print(X.shape, y.shape)
from itertools import islice
 
 
def chunk(arr_range, arr_size):
    arr_range = iter(arr_range)
    return iter(lambda: tuple(islice(arr_range, arr_size)), ())

def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num
 
X_New = list(chunk(X, 4000))[:-1]
y_New = list(chunk(y, 4000))[:-1]

for i in range(len(y_New)):
    y_New[i] = y_New[i][-1]


X = np.array(X_New)
y = np.array(y_New)

print(X.shape, y.shape)



(432774, 7) (432774, 5)
(108, 4000, 7) (108, 5)


In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras import layers

In [11]:
def build_model():
    
    inputs = layers.Input(shape=(4000, 7))
    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(inputs)
    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(x)
    x = layers.LSTM(64)(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    # x = layers.Softmax()(x)
    outputs = layers.Dense(5, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs], name='HAR')
    
    return model

model = build_model()

In [12]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=METRICS)

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    verbose=1,
    patience=4,
    mode='min',
    restore_best_weights=True)

In [16]:
history = model.fit(x = X, y= y, epochs=5, callbacks=[early_stopping])

Epoch 1/5
4/4 [==============================] - 6s 2s/step - loss: 0.6890 - tp: 51.0000 - fp: 165.0000 - tn: 267.0000 - fn: 57.0000 - accuracy: 0.4944 - precision: 0.2361 - recall: 0.4722 - auc: 0.5464 - prc: 0.2414
Epoch 2/5
4/4 [==============================] - 5s 1s/step - loss: 0.6832 - tp: 51.0000 - fp: 165.0000 - tn: 267.0000 - fn: 57.0000 - accuracy: 0.4944 - precision: 0.2361 - recall: 0.4722 - auc: 0.5293 - prc: 0.2119
Epoch 3/5
4/4 [==============================] - 5s 1s/step - loss: 0.6753 - tp: 51.0000 - fp: 165.0000 - tn: 267.0000 - fn: 57.0000 - accuracy: 0.4944 - precision: 0.2361 - recall: 0.4722 - auc: 0.5376 - prc: 0.2319
Epoch 4/5
4/4 [==============================] - 5s 1s/step - loss: 0.6683 - tp: 51.0000 - fp: 165.0000 - tn: 267.0000 - fn: 57.0000 - accuracy: 0.4944 - precision: 0.2361 - recall: 0.4722 - auc: 0.5367 - prc: 0.2318
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 0.6599 - tp: 51.0000 - fp: 165.0000 - tn: 267.0000 - fn: 57.0000

In [ ]:
model.save('HAR')

In [ ]:
model.evaluate(eval_ds)

43/43 [==============================] - 8s 69ms/step - loss: 0.5017 - tp: 1029.0000 - fp: 96.0000 - tn: 961.0000 - fn: 665.0000 - accuracy: 0.7234 - precision: 0.9147 - recall: 0.6074 - auc: 0.7925 - prc: 0.8843


[0.5017135739326477,
 1029.0,
 96.0,
 961.0,
 665.0,
 0.723373293876648,
 0.9146666526794434,
 0.6074380278587341,
 0.7924591302871704,
 0.8843364715576172]

In [ ]:
model.summary()

Model: "HAR"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 7)]          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 64)           10240     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 64)           24832     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0       